In [13]:
import sys
import torch
sys.path.append("/Users/philippvonbachmann/Documents/University/WiSe2122/ResearchProject/ResearchProjectLLVI/BasicExample")
from src.log_likelihood import LogLikelihoodMonteCarlo
from datasets.Regression.toydataset import create_dataset, sinus_mapping, dataset_to_loader, visualize_predictions

torch.manual_seed(5)

data_noise = 0.2
x_train, y_train, x_test, y_test = create_dataset(lower=-5, upper=7, mapping=sinus_mapping,cluster_pos=[-0.5,2], data_noise=data_noise, n_datapoints=256)

lr = 1e-4

from src.network.Regression import LLVIRegression
batch_size = 16
random_permutation = torch.randperm(len(x_train))
x_batch = torch.split(torch.unsqueeze(x_train[random_permutation], dim=1), batch_size)
y_batch =  torch.split(torch.unsqueeze(y_train[random_permutation], dim=1), batch_size)
train_set, test_set = dataset_to_loader(x_train, y_train, x_test, y_test , batch_size=16)

In [14]:
# from src.network.feature_extractor import FC_Net_2
from backpack import backpack, extend
laplace_model = torch.nn.Sequential(
        torch.nn.Linear(1, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 1)
)
loss_fun = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.Adam(laplace_model.parameters())
extend(laplace_model)
extend(loss_fun);

In [15]:
loss = loss_fun(laplace_model(torch.unsqueeze(x_train, dim=-1)), torch.unsqueeze(y_train, dim=-1))

In [16]:
loss

tensor(0.5232, grad_fn=<BackwardHookFunctionBackward>)

In [17]:
from backpack.extensions import (
    GGNMP,
    HMP,
    KFAC,
    KFLR,
    KFRA,
    PCHMP,
    BatchDiagGGNExact,
    BatchDiagGGNMC,
    BatchDiagHessian,
    BatchGrad,
    BatchL2Grad,
    DiagGGNExact,
    DiagGGNMC,
    DiagHessian,
    SqrtGGNExact,
    SqrtGGNMC,
    SumGradSquared,
    Variance,
)

In [18]:
from torch.autograd import grad
with backpack(DiagGGNExact()):
    grads = grad(loss, laplace_model.parameters(), create_graph=True, retain_graph=True)[0]

In [19]:
# with backpack(DiagGGNExact()):
#     loss.backward(create_graph=True)
# for name, param in laplace_model.named_parameters():
#     # print(name)
#     print(".grad.shape:             ", param.grad)
#     print(".diag_h.shape:           ", param.diag_h, name)
#     break
#     # print(".diag_h_batch.shape:     ", param.diag_h_batch.shape)

In [20]:
elbo = laplace_model[0].weight.diag_ggn_exact.mean()

In [21]:
elbo

tensor(0.0076, grad_fn=<MeanBackward0>)

In [10]:
from torch.autograd import grad

In [22]:
g = grad(elbo, laplace_model.parameters(), allow_unused=True)[0]

In [25]:
1000 * torch.nan

nan

In [23]:
print(g)

None


In [10]:
elbo.backward()

In [12]:
print(laplace_model[0].weight.grad)

None


In [11]:
optimizer.zero_grad()